In [1]:
# importing useful libs on the go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from xgboost.sklearn import XGBClassifier
import math
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import collections

plt.style.use('fivethirtyeight') # Good looking plots
pd.set_option('display.max_columns', None) # Display any number of columns

% matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test = pd.read_csv('test.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
trainc = train.copy()

In [5]:
(train.isnull().sum() == 300000).value_counts() #drop 7 directly

False    326
True       7
dtype: int64

In [9]:
lis = []
for col in list(train):
    if train[col].isnull().sum() == 300000:
        print col

EEG_TAG
EEG_CLOSED
PM_FD_MON_02
PM_FD_MON_04
STMT_CON_DAE_ACTIVE_MON_01
STMT_CON_DAE_CLOSED_MON_01
MER_EMI_CLOSED_MON_01


In [10]:
lis_drop = ['EEG_TAG','EEG_CLOSED', 'PM_FD_MON_02', 'PM_FD_MON_04', 'STMT_CON_DAE_ACTIVE_MON_01', 'STMT_CON_DAE_CLOSED_MON_01', 'MER_EMI_CLOSED_MON_01']

In [5]:
train.dropna(axis=1, how='all', inplace=True)

In [136]:
test.dropna(axis=1, how='all', inplace=True)

In [20]:
(train.isnull().sum() == 300000).value_counts() #dropped all nulls

False    326
dtype: int64

In [6]:
#dropping unnecessary features
train.drop(['CUSTOMER_ID'],axis=1,inplace=True)

In [11]:
train['LEGAL_ENTITY'].value_counts() #interesting!! -_-  remove FIRM, ASSOCIATION, NON_GOVERNMENTAL_ORG

INDIVIDUAL              299710
OTHERS                     283
NON_GOVERNMENTAL_ORG         4
ASSOCIATION                  2
FIRM                         1
Name: LEGAL_ENTITY, dtype: int64

In [7]:
train = train[train.LEGAL_ENTITY != 'FIRM']
train = train[train.LEGAL_ENTITY != 'ASSOCIATION']
train = train[train.LEGAL_ENTITY != 'NON_GOVERNMENTAL_ORG']

In [9]:
train.drop(['OCCUP_ALL_NEW'], axis=1, inplace=True) #all SALARIED

In [72]:
train.dtypes.unique()

array([dtype('O'), dtype('int64'), dtype('float64')], dtype=object)

In [31]:
train['AGE'].value_counts() # Hilarious !!! 117 age 127 people and 217 age is fucking 1173 people!! :P .... cool!! 

26     17479
27     17117
25     16556
24     16318
28     15967
29     15247
30     14165
31     13405
23     13386
32     12653
33     11530
22     10821
34     10661
35      9271
36      8817
37      7854
21      6968
38      6926
39      6032
40      5970
41      5638
42      5173
43      4591
44      4340
45      3880
46      3859
47      3536
48      3266
49      3017
51      2760
50      2746
52      2381
53      2163
54      2010
55      1715
56      1703
57      1621
58      1349
59      1255
217     1173
60      1026
61       963
62       821
63       683
64       626
65       428
117      127
Name: AGE, dtype: int64

In [57]:
train['AGE'] = trainc['AGE']

In [10]:
#grouping age
train['AGE'] = pd.cut(
    train['AGE'],
    [20, 23, 25, 27, 29, 31, 33, 36, 39, 43, 50, 300],
    labels=[1,2,3,4,5,6,7,8,9,10,11])

In [11]:
train['AGE'].value_counts()

3     34596
2     32874
4     31214
1     31175
7     28749
5     27570
10    24644
6     24183
11    22804
9     21372
8     20812
Name: AGE, dtype: int64

In [12]:
train['N_CASA_BAL_RANGE'] = train['N_CASA_MAX_BALANCE_MTD'] - train['N_CASA_MIN_BALANCE_MTD']

In [13]:
train.drop(['N_CASA_MAX_BALANCE_MTD','N_CASA_MIN_BALANCE_MTD'], axis=1, inplace=True)

In [108]:
300000 - train[['BL_TAG','TWL_TAG','PL_TAG','AL_CNC_TAG','AGRI_TAG','CE_TAG','CV_TAG','EDU_TAG','TL_TAG','OTHER_LOANS_TAG','LAP_TAG','LAS_TAG','EEG_TAG','BBG_TAG','HL_TAG','GL_TAG']].isnull().sum()

BL_TAG                71
TWL_TAG             4904
PL_TAG             17809
AL_CNC_TAG           584
AGRI_TAG              25
CE_TAG                 8
CV_TAG                25
EDU_TAG               52
TL_TAG                23
OTHER_LOANS_TAG      432
LAP_TAG               70
LAS_TAG               36
EEG_TAG                7
BBG_TAG                9
HL_TAG              2635
GL_TAG               773
dtype: int64

In [14]:
train['LIVE_TAG'] = 0

In [15]:
lis1 = ['BL_TAG','TWL_TAG','PL_TAG','AL_CNC_TAG','AGRI_TAG','CE_TAG','CV_TAG','EDU_TAG','TL_TAG','OTHER_LOANS_TAG','LAP_TAG','LAS_TAG','BBG_TAG','HL_TAG','GL_TAG']

In [17]:
for col in lis1:
    train[col].fillna(0, inplace=True)
    
for column in lis1:
    train[column].replace("Y", 1, inplace=True)
    
for col in lis1:
    train['LIVE_TAG'] += train[col]

In [20]:
train.drop(lis1, axis=1, inplace=True)

In [110]:
300000 - train[['AL_CLOSED','PL_CLOSED','BL_CLOSED','TWL_CLOSED','CV_CLOSED','LAP_CLOSED','LAS_CLOSED','EEG_CLOSED','BBG_CLOSED','GL_CLOSED']].isnull().sum()

AL_CLOSED     1262
PL_CLOSED     7327
BL_CLOSED       55
TWL_CLOSED    2409
CV_CLOSED       30
LAP_CLOSED      76
LAS_CLOSED      37
EEG_CLOSED       7
BBG_CLOSED       9
GL_CLOSED     1143
dtype: int64

In [22]:
train['CLOSED_LOAN'] = 0

In [23]:
lis2 = ['AL_CLOSED','PL_CLOSED','BL_CLOSED','TWL_CLOSED','CV_CLOSED','LAP_CLOSED','LAS_CLOSED','BBG_CLOSED','GL_CLOSED']

In [24]:
for col in lis2:
    train[col].fillna(0, inplace=True)

for column in lis2:
    train[column].replace("Y", 1, inplace=True)
    
for col in lis2:
    train['CLOSED_LOAN'] += train[col]

In [25]:
train.drop(lis2, axis=1, inplace=True)

In [26]:
lis3 = ['CC_HOLD_MON_01','CC_HOLD_MON_02','CC_HOLD_MON_03']
lis4 = ['CC_ACTIVE_MON_01','CC_ACTIVE_MON_02','CC_ACTIVE_MON_03']

In [27]:
train['CC_HOLD_MON'] = 0
train['CC_ACTIVE_MON'] = 0

In [28]:
for col in lis3:
    train[col].fillna(0, inplace=True)

for column in lis3:
    train[column].replace("Y", 1, inplace=True)
    
for col in lis3:
    train['CC_HOLD_MON'] += train[col]
    
train.drop(lis3, axis=1, inplace=True)

In [29]:
for col in lis4:
    train[col].fillna(0, inplace=True)

for column in lis4:
    train[column].replace("Y", 1, inplace=True)
    
for col in lis4:
    train['CC_ACTIVE_MON'] += train[col]
    
train.drop(lis4, axis=1, inplace=True)

In [30]:

lis5 = ['DC_ACTIVE_MON_01','DC_ACTIVE_MON_02','DC_ACTIVE_MON_03']

train['DC_ACTIVE_MON'] = 0

for col in lis5:
    train[col].fillna(0, inplace=True)

for column in lis5:
    train[column].replace("Y", 1, inplace=True)
    
for col in lis5:
    train['DC_ACTIVE_MON'] += train[col]
    
train.drop(lis5, axis=1, inplace=True)

In [31]:
dic = {'Y' :1, 'N' :0}
lis = ['CC_HOLD','CC_ACTIVE','DC_HOLD','DC_ACTIVE']
for col in lis: 
    train[col] = train[col].map(dic).astype(int)

In [34]:
train['DC_SPEND_MON_01_'] = 0.0

In [35]:
lis = ['DC_SPEND_MON_01_ENT', 'DC_SPEND_MON_01_MED', 'DC_SPEND_MON_01_HMD', 'DC_SPEND_MON_01_HBY', 'DC_SPEND_MON_01_CARE', 'DC_SPEND_MON_01_TRL', 'DC_SPEND_MON_01_RST', 'DC_SPEND_MON_01_JER', 'DC_SPEND_MON_01_HTL', 'DC_SPEND_MON_01_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

In [37]:
for col in lis:
    train['DC_SPEND_MON_01_'] += train[col]

In [42]:
trainc = train.copy()

In [45]:
train.drop(lis, axis=1, inplace=True)

In [47]:
train['DC_SPEND_MON_02_'] = 0.0

lis = ['DC_SPEND_MON_02_ENT', 'DC_SPEND_MON_02_MED', 'DC_SPEND_MON_02_HMD', 'DC_SPEND_MON_02_HBY', 'DC_SPEND_MON_02_CARE', 'DC_SPEND_MON_02_TRL', 'DC_SPEND_MON_02_RST', 'DC_SPEND_MON_02_JER', 'DC_SPEND_MON_02_HTL', 'DC_SPEND_MON_02_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

for col in lis:
    train['DC_SPEND_MON_02_'] += train[col]

train.drop(lis, axis=1, inplace=True)



In [49]:
train['DC_SPEND_MON_03_'] = 0.0

lis = ['DC_SPEND_MON_03_ENT', 'DC_SPEND_MON_03_MED', 'DC_SPEND_MON_03_HMD', 'DC_SPEND_MON_03_HBY', 'DC_SPEND_MON_03_CARE', 'DC_SPEND_MON_03_TRL', 'DC_SPEND_MON_03_RST', 'DC_SPEND_MON_03_JER', 'DC_SPEND_MON_03_HTL', 'DC_SPEND_MON_03_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

for col in lis:
    train['DC_SPEND_MON_03_'] += train[col]

train.drop(lis, axis=1, inplace=True)



In [50]:
train['DC_SPEND_MON_05_'] = 0.0

lis = ['DC_SPEND_MON_05_ENT', 'DC_SPEND_MON_05_MED', 'DC_SPEND_MON_05_HMD', 'DC_SPEND_MON_05_HBY', 'DC_SPEND_MON_05_CARE', 'DC_SPEND_MON_05_TRL', 'DC_SPEND_MON_05_RST', 'DC_SPEND_MON_05_JER', 'DC_SPEND_MON_05_HTL', 'DC_SPEND_MON_05_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

for col in lis:
    train['DC_SPEND_MON_05_'] += train[col]

train.drop(lis, axis=1, inplace=True)



In [51]:
train['DC_SPEND_MON_04_'] = 0.0

lis = ['DC_SPEND_MON_04_ENT', 'DC_SPEND_MON_04_MED', 'DC_SPEND_MON_04_GRC', 'DC_SPEND_MON_04_HMD', 'DC_SPEND_MON_04_HBY', 'DC_SPEND_MON_04_CARE', 'DC_SPEND_MON_04_TRL', 'DC_SPEND_MON_04_RST', 'DC_SPEND_MON_04_JER', 'DC_SPEND_MON_04_HTL', 'DC_SPEND_MON_04_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

for col in lis:
    train['DC_SPEND_MON_04_'] += train[col]

train.drop(lis, axis=1, inplace=True)



In [53]:
train['CC_SPEND_MON_02_'] = 0.0

lis = ['CC_SPEND_MON_02_ENT', 'CC_SPEND_MON_02_MED', 'CC_SPEND_MON_02_HMD', 'CC_SPEND_MON_02_HBY', 'CC_SPEND_MON_02_CARE', 'CC_SPEND_MON_02_TRL', 'CC_SPEND_MON_02_RST', 'CC_SPEND_MON_02_JER', 'CC_SPEND_MON_02_HTL', 'CC_SPEND_MON_02_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

for col in lis:
    train['CC_SPEND_MON_02_'] += train[col]

train.drop(lis, axis=1, inplace=True)



In [54]:
train['CC_SPEND_MON_01_'] = 0.0

lis = ['CC_SPEND_MON_01_ENT', 'CC_SPEND_MON_01_MED', 'CC_SPEND_MON_01_HMD', 'CC_SPEND_MON_01_HBY', 'CC_SPEND_MON_01_CARE', 'CC_SPEND_MON_01_TRL', 'CC_SPEND_MON_01_RST', 'CC_SPEND_MON_01_JER', 'CC_SPEND_MON_01_HTL', 'CC_SPEND_MON_01_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

for col in lis:
    train['CC_SPEND_MON_01_'] += train[col]

train.drop(lis, axis=1, inplace=True)



In [55]:
train['CC_SPEND_MON_03_'] = 0.0

lis = ['CC_SPEND_MON_03_ENT', 'CC_SPEND_MON_03_MED', 'CC_SPEND_MON_03_HMD', 'CC_SPEND_MON_03_HBY', 'CC_SPEND_MON_03_CARE', 'CC_SPEND_MON_03_TRL', 'CC_SPEND_MON_03_RST', 'CC_SPEND_MON_03_JER', 'CC_SPEND_MON_03_HTL', 'CC_SPEND_MON_03_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

for col in lis:
    train['CC_SPEND_MON_03_'] += train[col]

train.drop(lis, axis=1, inplace=True)



In [56]:
train['CC_SPEND_MON_04_'] = 0.0

lis = ['CC_SPEND_MON_04_ENT', 'CC_SPEND_MON_04_MED', 'CC_SPEND_MON_04_HMD', 'CC_SPEND_MON_04_HBY', 'CC_SPEND_MON_04_CARE', 'CC_SPEND_MON_04_TRL', 'CC_SPEND_MON_04_RST', 'CC_SPEND_MON_04_JER', 'CC_SPEND_MON_04_HTL', 'CC_SPEND_MON_04_ATM']

for col in lis:
    train[col].fillna(0.0, inplace=True)

for col in lis:
    train['CC_SPEND_MON_04_'] += train[col]

train.drop(lis, axis=1, inplace=True)



In [67]:
lis = ['DC_SPEND_MON_01', 'DC_SPEND_MON_02', 'DC_SPEND_MON_03', 'DC_SPEND_MON_04', 'DC_SPEND_MON_05', 'DC_SPEND_MON_06', 'CC_SPEND_MON_01', 'CC_SPEND_MON_02', 'CC_SPEND_MON_03', 'CC_SPEND_MON_04', 'CC_SPEND_MON_05', 'CC_SPEND_MON_06', 'DC_TXN_MON_01', 'DC_TXN_MON_02', 'DC_TXN_MON_03', 'DC_TXN_MON_04', 'DC_TXN_MON_05', 'DC_TXN_MON_06', 'CC_TXN_MON_01', 'CC_TXN_MON_02', 'CC_TXN_MON_03', 'CC_TXN_MON_04', 'CC_TXN_MON_05', 'CC_TXN_MON_06']

In [73]:
for col in lis:
    train[col].fillna(0.0, inplace=True)

In [74]:
train['DC_SPEND_MON'] = train['DC_SPEND_MON_01'] + train['DC_SPEND_MON_02'] + train['DC_SPEND_MON_03'] + train['DC_SPEND_MON_04'] + train['DC_SPEND_MON_05'] + train['DC_SPEND_MON_06']

In [76]:
train.drop(['DC_SPEND_MON_01','DC_SPEND_MON_02','DC_SPEND_MON_03','DC_SPEND_MON_04','DC_SPEND_MON_05','DC_SPEND_MON_06'], axis=1, inplace=True)

In [78]:
train['CC_SPEND_MON'] = train['CC_SPEND_MON_01'] + train['CC_SPEND_MON_02'] + train['CC_SPEND_MON_03'] + train['CC_SPEND_MON_04'] + train['CC_SPEND_MON_05'] + train['CC_SPEND_MON_06']

In [79]:
train.drop(['CC_SPEND_MON_01','CC_SPEND_MON_02','CC_SPEND_MON_03','CC_SPEND_MON_04','CC_SPEND_MON_05','CC_SPEND_MON_06'], axis=1, inplace=True)

In [81]:
train['DC_SPEND_MON_'] = train['DC_SPEND_MON_01_'] + train['DC_SPEND_MON_02_'] + train['DC_SPEND_MON_03_'] + train['DC_SPEND_MON_04_'] + train['DC_SPEND_MON_05_'] 
train.drop(['DC_SPEND_MON_01_','DC_SPEND_MON_02_','DC_SPEND_MON_03_','DC_SPEND_MON_04_','DC_SPEND_MON_05_'], axis=1, inplace=True)

In [84]:
train['CC_SPEND_MON_'] = train['CC_SPEND_MON_01_'] + train['CC_SPEND_MON_02_'] + train['CC_SPEND_MON_03_'] + train['CC_SPEND_MON_04_'] 
train.drop(['CC_SPEND_MON_01_','CC_SPEND_MON_02_','CC_SPEND_MON_03_','CC_SPEND_MON_04_'], axis=1, inplace=True)

In [86]:
train['DC_TXN_MON'] = train['DC_TXN_MON_01'] + train['DC_TXN_MON_02'] + train['DC_TXN_MON_03'] + train['DC_TXN_MON_04'] + train['DC_TXN_MON_05'] + train['DC_TXN_MON_06']  
train.drop(['DC_TXN_MON_01','DC_TXN_MON_02','DC_TXN_MON_03','DC_TXN_MON_04','DC_TXN_MON_05', 'DC_TXN_MON_06'], axis=1, inplace=True)

In [87]:
train['CC_TXN_MON'] = train['CC_TXN_MON_01'] + train['CC_TXN_MON_02'] + train['CC_TXN_MON_03'] + train['CC_TXN_MON_04'] + train['CC_TXN_MON_05'] + train['CC_TXN_MON_06']  
train.drop(['CC_TXN_MON_01','CC_TXN_MON_02','CC_TXN_MON_03','CC_TXN_MON_04','CC_TXN_MON_05', 'CC_TXN_MON_06'], axis=1, inplace=True)

In [107]:
train['PM_FD_MON_01'].fillna(0, inplace=True)

In [120]:
train['STMT_MON_01'].isnull().sum()

216993

In [123]:
train['STMT_MON_01']

216993

In [124]:
train['RESPONDERS'].value_counts()

N    295381
Y      4612
Name: RESPONDERS, dtype: int64

In [134]:
train.shape

(299993, 187)

In [22]:
(train.isnull().sum()/300000 > .8).sum()

226

In [21]:
(train.isnull().sum()/300000 > .9).sum()

205

In [24]:
(train.isnull().sum()/300000 > .96).sum()

173

In [25]:
(train.isnull().sum()/300000 > .99).sum()

101

In [129]:
test['RESPONDERS'] = 0

In [130]:
test.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,EEG_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,EEG_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,PM_FD_MON_02,PM_FD_MON_04,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,MATURITY_GL,MATURITY_LAP,MATURITY_LAS,STMT_CON_DAE_ACTIVE_MON_01,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,STMT_CON_DAE_CLOSED_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,MER_EMI_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_

In [131]:
ans = test[['CUSTOMER_ID', 'RESPONDERS']]

In [132]:
ans.to_csv('H.O.U.N.D_1.csv', index=False)